# Data preprocessing

Первичная обработка и подготовка данных для дальнейших исследований

In [ ]:
import os
import pandas as pd

os.chdir("../")

from src.parsing.moex import read_moex_dynamics_data

## Dynamics

In [2]:
df_dynamics = read_moex_dynamics_data()
df_dynamics = df_dynamics.rename(columns={"secId": "ticket"})
df_dynamics.head(3)

100%|█████████████████████████████████████████| 452/452 [00:10<00:00, 41.52it/s]


,id,ticket,board,boardGroup,shortName,isin,type,listLevel,date,last,...,open,high,low,waPrice,volume,numTrades,volumeCount,issuerName,capitalization,emitterId
0,444,KLSB,TQBR,MAIN,КалужскСК,RU000A0DKZK3,common_share,3.0,2025-07-03,21.84,...,21.70,21.92,21.68,21.82,1119304.0,94,51300,"ПАО ""Калужская сбытовая компания""",1.998084e+09,945
1,444,KLSB,TQBR,MAIN,КалужскСК,RU000A0DKZK3,common_share,3.0,2025-07-02,21.69,...,22.11,22.17,21.69,21.89,1188561.0,298,54300,"ПАО ""Калужская сбытовая компания""",1.984361e+09,945
2,444,KLSB,TQBR,MAIN,КалужскСК,RU000A0DKZK3,common_share,3.0,2025-07-01,21.99,...,22.03,22.30,21.99,22.08,1346995.0,196,61000,"ПАО ""Калужская сбытовая компания""",2.011807e+09,945


**Описание итоговых фичей:**
- **id** - уникальный идентификатора
- **ticket** - тикет на бирже
- **boardGroup** - режим торгов
- **shortName** - название компании
- **type** - тип инструмента
- **listLevel** - уровень листинга
- **date** - дата торгов
- **volumeCount** - объем торгов
- **capitalization** - капитализация
- **low** - минимальная цена
- **high** - максимальная цена
- **waPrice** - средневзвешенная цена

In [3]:
columns =[
    "id", "ticket", "boardGroup",
    "shortName", "type", "listLevel",
    "date", "volumeCount", "capitalization",
    "low", "high", "waPrice"
]
df_dynamics_preprocessed = df_dynamics[columns]
df_dynamics_preprocessed.head()

,id,ticket,boardGroup,shortName,type,listLevel,date,volumeCount,capitalization,low,high,waPrice
0,444,KLSB,MAIN,КалужскСК,common_share,3.0,2025-07-03,51300,1.998084e+09,21.68,21.92,21.82
1,444,KLSB,MAIN,КалужскСК,common_share,3.0,2025-07-02,54300,1.984361e+09,21.69,22.17,21.89
2,444,KLSB,MAIN,КалужскСК,common_share,3.0,2025-07-01,61000,2.011807e+09,21.99,22.30,22.08
3,444,KLSB,MAIN,КалужскСК,common_share,3.0,2025-06-30,41500,2.018211e+09,21.75,22.32,22.04
4,444,KLSB,MAIN,КалужскСК,common_share,3.0,2025-06-27,8300,2.014551e+09,21.90,22.15,22.02


In [8]:
print(f"rows amount: {df_dynamics_preprocessed.shape[0]}")
print(f"tickets amount: {df_dynamics_preprocessed.ticket.nunique()}")

rows amount: 458519
tickets amount: 452


In [9]:
# Сохранение итогового датафрейма с динамикой стоимости инструментов
df_dynamics_preprocessed.to_csv("./data/dynamics.csv", index=False)

## Dividends

In [114]:
df_dividends = pd.read_csv("./data/raw_dividends_data.csv")
df_dividends.head()

,period,amount,company_short_name,company_name
0,закрытие реестра 7.07.2025 2024,"5,01 руб.",abrau-dyurso,Абрау-Дюро(ABRD)
1,закрытие реестра 17.07.2024 2023,"3,27 руб.",abrau-dyurso,Абрау-Дюро(ABRD)
2,закрытие реестра 17.07.2023 2023 I квартал,"6,14 руб.",abrau-dyurso,Абрау-Дюро(ABRD)
3,закрытие реестра 17.07.2023 2022,"0,19 руб.",abrau-dyurso,Абрау-Дюро(ABRD)
4,закрытие реестра 8.07.2022 2021,"3,44 руб.",abrau-dyurso,Абрау-Дюро(ABRD)


In [115]:
# Извлечение суммы
df_dividends.amount = df_dividends.amount.apply(lambda x: x.replace("$", "USD"))
df_dividends["dividend_amount"] = df_dividends.amount.str.extract(r'(\d+(?:[.,]\d{2})?)')
df_dividends["currency"] = df_dividends.amount.str.extract(r'([a-zA-Zа-яА-Я]+)')
df_dividends.head()

,period,amount,company_short_name,company_name,dividend_amount,currency
0,закрытие реестра 7.07.2025 2024,"5,01 руб.",abrau-dyurso,Абрау-Дюро(ABRD),"5,01",руб
1,закрытие реестра 17.07.2024 2023,"3,27 руб.",abrau-dyurso,Абрау-Дюро(ABRD),"3,27",руб
2,закрытие реестра 17.07.2023 2023 I квартал,"6,14 руб.",abrau-dyurso,Абрау-Дюро(ABRD),"6,14",руб
3,закрытие реестра 17.07.2023 2022,"0,19 руб.",abrau-dyurso,Абрау-Дюро(ABRD),"0,19",руб
4,закрытие реестра 8.07.2022 2021,"3,44 руб.",abrau-dyurso,Абрау-Дюро(ABRD),"3,44",руб


In [116]:
# Извлечения года выплаты дивидендов
df_dividends["year"] = df_dividends.period.str.extract(r"(\d{4})")
df_dividends.head()

,period,amount,company_short_name,company_name,dividend_amount,currency,year
0,закрытие реестра 7.07.2025 2024,"5,01 руб.",abrau-dyurso,Абрау-Дюро(ABRD),"5,01",руб,2025
1,закрытие реестра 17.07.2024 2023,"3,27 руб.",abrau-dyurso,Абрау-Дюро(ABRD),"3,27",руб,2024
2,закрытие реестра 17.07.2023 2023 I квартал,"6,14 руб.",abrau-dyurso,Абрау-Дюро(ABRD),"6,14",руб,2023
3,закрытие реестра 17.07.2023 2022,"0,19 руб.",abrau-dyurso,Абрау-Дюро(ABRD),"0,19",руб,2023
4,закрытие реестра 8.07.2022 2021,"3,44 руб.",abrau-dyurso,Абрау-Дюро(ABRD),"3,44",руб,2022


In [133]:
# Извлечение тикета
df_dividends.company_name = df_dividends.company_name.str.upper()
df_dividends["ticket"] = df_dividends.company_name.str.extract(r"\(([A-Z]{4})\)")
df_dividends.head()

,period,amount,company_short_name,company_name,dividend_amount,currency,year,ticket
0,закрытие реестра 7.07.2025 2024,"5,01 руб.",abrau-dyurso,АБРАУ-ДЮРО(ABRD),"5,01",руб,2025,ABRD
1,закрытие реестра 17.07.2024 2023,"3,27 руб.",abrau-dyurso,АБРАУ-ДЮРО(ABRD),"3,27",руб,2024,ABRD
2,закрытие реестра 17.07.2023 2023 I квартал,"6,14 руб.",abrau-dyurso,АБРАУ-ДЮРО(ABRD),"6,14",руб,2023,ABRD
3,закрытие реестра 17.07.2023 2022,"0,19 руб.",abrau-dyurso,АБРАУ-ДЮРО(ABRD),"0,19",руб,2023,ABRD
4,закрытие реестра 8.07.2022 2021,"3,44 руб.",abrau-dyurso,АБРАУ-ДЮРО(ABRD),"3,44",руб,2022,ABRD


In [140]:
# Формирование итогового датафрейма с дивидендами и удаление пустых строк
df_dividends_preprocessed = df_dividends[["ticket", "year", "dividend_amount", "currency"]]
df_dividends_preprocessed = df_dividends_preprocessed.dropna()
df_dividends_preprocessed.head()

,ticket,year,dividend_amount,currency
0,ABRD,2025,"5,01",руб
1,ABRD,2024,"3,27",руб
2,ABRD,2023,"6,14",руб
3,ABRD,2023,"0,19",руб
4,ABRD,2022,"3,44",руб


In [148]:
print(f"rows amount: {df_dividends_preprocessed.shape[0]}")
print(f"tickets amount: {df_dividends_preprocessed.ticket.nunique()}")

rows amount: 2893
tickets amount: 254


In [149]:
# Сохранение итогового датафрейма с дивидендами
df_dividends_preprocessed.to_csv("./data/dividends.csv")